<a href="https://colab.research.google.com/github/futugyou/pyproject/blob/master/transformers_demo/text_generation_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required dependencies
%pip install langchain-community langchain-google-genai langchain google-generativeai
# Goal: Generate title, characters, story outline

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",api_key=userdata.get('GOOGLE_API_KEY'))

In [ ]:
# 1. generate title
from langchain import PromptTemplate, LLMChain

template = """<s><|user|>
create a title for a story about {summary}. only return the title.<|end|>
<|assistant|>
"""

title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title_chain = (title_prompt | llm).with_config(run_name="title")

response = title_chain.invoke(
    {
        "summary": "a girl that lost her mother",
    }
)
print(response.content)